# AUTO$^2$ and AUTO-Demos : *plp* - Fold Continuation for Periodic Solutions.

**This is an example on how to use AUTO$^2$ to explore the AUTO Demos *plp* bifurcations and solutions.**

In this example, a one-compartment activator-inhibitor system (Kernévez (1980)) is investigated. The equations of the model are given by:

$$
\dot s = (s_0 - s) - \rho \, R(s, a)
$$
$$
\dot a = \alpha (a_0 - a) - \rho \, R(s, a)
$$

where $R(s) = (s \ a) / (1 + s + \kappa\, s^2)$ with $\kappa > 0$. The free parameter is $\rho$ . Other parameters are fixed. This equation is also considered in Doedel, Keller & Kernévez (1991a).

We are thus going to find the fixed points and periodic orbits of this system and continue them by varying $\rho$.

#### References

* Kernévez, J. P. (1980), *Enzyme Mathematics*, North-Holland Press, Amsterdam. [[link](https://books.google.be/books?hl=fr&lr=&id=04n1XU16khIC&oi=fnd&pg=PP1&dq=kernevez+enzyme+mathematics&ots=X_64rEJO8t&sig=nOF7rxGrchs3L5vGbpPgKHLyJ28#v=onepage&q=kernevez%20enzyme%20mathematics&f=false)]
* Doedel, E. J., Keller, H. B. & Kernévez, J. P. (1991a), ‘Numerical analysis and control of
bifurcation problems: (I) Bifurcation in finite dimensions’, *Int. J. Bifurcation and Chaos*
**1**(3), 493–520. [[link](https://www.worldscientific.com/doi/abs/10.1142/s0218127491000397)]

## Code

First we set the Python path if needed:

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../../')])

And load the needed libraries, including AUTO$^2$:

In [ ]:
import numpy as np
from numba import njit
from scipy.optimize import root
from scipy.integrate import solve_ivp

In [ ]:
from auto2.diagrams.bifurcations import BifurcationDiagram

Creating the model equations

In [ ]:
# maybe finally not needed here

@njit
def plp(u, s0, a0, alpha, rho, kappa):
    F = np.zeros(2)
    s = u[0]
    a = u[1]
    d = 1 + s + kappa * s**2
    r = s * a / d
    F[1] = s0 - s - rho * r
    F[2] = alpha * (a0 - a) - rho * r

    return F

@njit
def plpt(t, u, s0, a0, alpha, rho, kappa):
    return plp(u, s0, a0, alpha, rho, kappa)

and define a set of standard parameters:

In [ ]:
params = {
    's0': 110.,
    'a0': 500.,
    'alpha': 0.2,
    'rho': 2.021628,
    'kappa': 0.1,
}

Next we setup the list of points and parameters that AUTO$^2$ will have to continue. We are providing only one point for the value $\rho = 2.021628$ :

In [ ]:
initial_points = list()

initial_points.append({'parameters': params, 'initial_data': np.array([4.555974e1, 1.777987e2])})

We are now ready to compute the diagram of fixed points as a function of $\rho$. Note that we specify that the bifurcation diagram object must load the ̀`plp.f90` and `c.plp` files where the model equations and continuation parameters have been written.

In [ ]:
b = BifurcationDiagram('plp')

b.compute_fixed_points_diagram(initial_points,extra_comparison_parameters=['s', 'a'], comparison_tol=[1.e-1] * 3,
                               ICP=['rho'], NMX=1000, UZSTOP={'rho':[2.,6.]}, UZR={'rho': list(np.arange(2.25, 6., 0.25))}, NPR=0)

We can now plot the result as functions of $\rho$ and $L^2$ norm :

In [ ]:
b.plot_fixed_points_diagram(cmap='gist_ncar');

and also as functions of $s$ and $a$ :

In [ ]:
b.plot_fixed_points_diagram((2,3),cmap='gist_ncar');

We see that two Hopf bifurcations were found.

We can continue periodic orbits out of these Hopf bifurcations : 

In [ ]:
b.compute_periodic_orbits_diagram(
    3, 
    extra_comparison_parameters=['s', 'a'], 
    comparison_tol=[1.e-1, 1.e-1, 1.e-1],
    ICP=['rho']
)


and plot the results on a bifurcation diagram in $L^2$ norm:

In [ ]:
ax = b.plot_fixed_points_diagram(cmap='gist_ncar')
b.plot_periodic_orbits_diagram(ax=ax);

or in the maximum of the $a$ variable:

In [ ]:
ax = b.plot_fixed_points_diagram(cmap='gist_ncar', variables=(0,3))
b.plot_periodic_orbits_diagram(ax=ax, variables=(0,3));

We can also plot both the bifurcation diagram and the solutions for a given value of $\rho$:

In [ ]:
b.plot_diagram_and_solutions(3., solutions_variables=(0, 1), diagram_variables=(3,), fixed_points_diagram_kwargs={'legend': True, 'cmap': 'gist_ncar'}, );

or plot a single branch and its solutions:

In [ ]:
# plotting branch 4
b.plot_single_po_branch_and_solutions(2, diagram_variables=(3,), cmap='Blues_r');

You can now explore continuations in other parameters: